In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1,920 kB]
Hit:13 http://pp

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-09 14:03:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.30MB/s    in 0.2s    

2023-02-09 14:03:36 (5.30 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [6]:
# 1) Filtered dataframe with total_votes greater or equal than 0
vine20_df = vine_df.filter(vine_df["total_votes"]>=20)
vine20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [7]:
# 2) Filtered dataframe helpful votes divided by total votes is equal to or greater than 50%
vine20_df = vine20_df.withColumn('division',vine20_df['helpful_votes']/vine20_df['total_votes'])

In [8]:
# 2) Filtered dataframe helpful votes divided by total votes is equal to or greater than 50%
vine50_df = vine20_df.filter(vine20_df["helpful_votes"] / vine20_df["total_votes"]>=0.5)
vine50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          division|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|0.7704918032786885|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|0.9130434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|0.9736842105263158|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|0.8918918918918919|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|0.9583333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y| 0.918918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|0.9565217391304348|
|R1P7GJ0IN2BRNH|          5|  

In [9]:
# 3) Filtered dataframe with Vine paid program
vineY_df = vine50_df.filter(vine50_df["verified_purchase"]=="Y")
vineY_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          division|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|0.9130434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|0.9736842105263158|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|0.9583333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y| 0.918918918918919|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|               1.0|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|              0.95|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|                Y|0.9545454545454546|
|R238ZSG6TSUBNX|          5|  

In [10]:
  # 4) Filtered dataframe with Vine not paid program
vineN_df = vine50_df.filter(vine50_df["verified_purchase"]=="N")
vineN_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          division|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|0.7704918032786885|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|0.8918918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|0.9565217391304348|
| RFL3TJED8C240|          5|           13|         24|   N|                N|0.5416666666666666|
|R2NTTIW5KBKLYG|          5|           19|         20|   N|                N|              0.95|
|R3JNDLZ3KACRLG|          5|           29|         33|   N|                N|0.8787878787878788|
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|0.8695652173913043|
|R2YYNS9L1J2365|          5|  

In [11]:
#Import functions
from pyspark.sql.functions import count
vine_df.select(count('review_id')).show()

+----------------+
|count(review_id)|
+----------------+
|          904765|
+----------------+



In [12]:
vineY_df.select(count('review_id')).show()

+----------------+
|count(review_id)|
+----------------+
|            8610|
+----------------+



In [13]:
vineN_df.select(count('review_id')).show()

+----------------+
|count(review_id)|
+----------------+
|            5927|
+----------------+



In [14]:
vineY_df.select('star_rating').where(vineY_df.star_rating==5).count()

4940

In [15]:
vineN_df.select('star_rating').where(vineN_df.star_rating==5).count()

3306

In [23]:
vineY_df.agg((vineY_df.select('star_rating').where(vineY_df.star_rating==5).count()) / count('review_id')).show()

+-------------------------+
|(4940 / count(review_id))|
+-------------------------+
|       0.5737514518002322|
+-------------------------+



In [24]:
vineN_df.agg((vineN_df.select('star_rating').where(vineN_df.star_rating==5).count()) / count('review_id')).show()

+-------------------------+
|(3306 / count(review_id))|
+-------------------------+
|       0.5577864012147798|
+-------------------------+

